# Promethium: Robust Auto-Fallback Loader

This notebook demonstrates a **universal loading pattern** for `promethium-seismic` that works across all environments:
1.  **Development**: Checks if already installed (e.g. via pip).
2.  **Offline Wheel**: Checks if a local wheel dataset is present and installs it.
3.  **Source Fallback**: Checks if a source dataset is present and injects it into path.

Use this pattern when you want your notebook to be portable between local dev, Kaggle online, and Kaggle offline modes.

---

## 1. Universal Import Function

Copy this function into the first cell of your notebook.

In [ ]:
import sys
import os
import subprocess
import importlib
from typing import Optional

def load_promethium() -> None:
    """
    Robustly loads the promethium library, trying multiple methods:
    1. Import installed package (pip)
    2. Install from local wheel dataset (offline)
    3. Import from source dataset (pip-less)
    """
    # 1. Try importing existing installation
    try:
        import promethium
        if promethium.__version__ < "1.0.3": # Optional: force update check
             pass
        else:
             print(f"[Mode: Installed] Loaded promethium {promethium.__version__}")
             return
    except ImportError:
        pass

    # 2. Try installing from local wheel dataset
    wheel_dir = "../input/promethium-seismic-wheel-100"  # Update version as needed
    if os.path.exists(wheel_dir):
        wheels = [f for f in os.listdir(wheel_dir) if f.endswith(".whl")]
        if wheels:
            wheel_path = os.path.join(wheel_dir, wheels[0])
            print(f"[Mode: Wheel] Found wheel: {wheel_path}. Installing...")
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", wheel_path, "--no-deps"])
                import promethium
                print(f"[Mode: Wheel] Successfully installed and loaded promethium {promethium.__version__}")
                return
            except Exception as e:
                print(f"[Mode: Wheel] Installation failed: {e}")
    
    # 3. Try injecting source path from dataset
    source_root = "/kaggle/input/promethium-seismic-source-100" # Update version as needed
    if os.path.exists(source_root):
        print(f"[Mode: Source] Found source dataset at {source_root}. Injecting path...")
        if source_root not in sys.path:
            sys.path.insert(0, source_root)
        
        try:
            import promethium
            print(f"[Mode: Source] Successfully loaded promethium {promethium.__version__}")
            return
        except ImportError as e:
            print(f"[Mode: Source] Import failed: {e}")

    raise ImportError("Could not load promethium-seismic. No installed package, wheel dataset, or source dataset found.")

# Execute the loader
load_promethium()

## 2. Usage

After running the loader, the library is ready to use.

In [ ]:
import promethium
from promethium import SeismicRecoveryPipeline

print(f"Library available at: {promethium.__file__}")